In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\real_estate_dataset.csv")

In [2]:
df.head()


,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,2,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,3,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,4,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,5,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523


In [3]:
df[df.isna().any(axis=1)]

,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price


In [4]:
traincol=df.columns
traincol=traincol.delete(loc=0)
traincol=traincol.delete(loc=-1)
X=df[traincol]
Y=df['Price']

In [7]:
import numpy as np
xarray=np.array(X)
yarray=np.array(Y)

In [8]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(xarray,yarray,test_size=0.3,random_state=69)
Xtest,Xval,ytest,yval=train_test_split(Xtest,ytest,test_size=0.5,random_state=69)


In [9]:
import torch
from torch.utils.data import DataLoader,Dataset

class dataset(Dataset):
    def __init__(self,X,Y):
        self.X=torch.tensor(X, dtype=torch.float32)
        self.Y=torch.tensor(Y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]
    
traindataset=dataset(X=Xtrain,Y=ytrain)
testdataset=dataset(X=Xtest,Y=ytest)
valdataset=dataset(X=Xval,Y=yval)



In [10]:
traindataloader=DataLoader(traindataset,batch_size=10,shuffle=True)
testdataloader=DataLoader(testdataset,batch_size=10,shuffle=True)
valdataloader=DataLoader(valdataset,batch_size=10,shuffle=True)

In [20]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Model,self).__init__(*args, **kwargs)
        self.input=nn.Linear(X.shape[1],20)
        self.output=nn.Linear(20,1)
        
    def forward(self,x):
        x=self.input(x)
        x=self.output(x)
        return x


model=Model()


In [16]:
from torchsummary import summary

summary(model,input_size=(10,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 20]             220
            Linear-2                    [-1, 1]              21
Total params: 241
Trainable params: 241
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [18]:
import torch.optim as optim
critetion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [23]:
epoch=10

for i in range(epoch):
    train_loss=0
    absolute_errors=0
    total_mae=0
    total_rmse=0
    avg_mae=0
    avg_rmse=0
    avg_train_loss=0
    ss_residual=0
    ss_total=0
    test_r2=0

    for data in traindataloader:
        values,lables=data
        prediction=model(values).squeeze(1)
        batchloss=critetion(prediction,lables)


        optimizer.zero_grad()
        batchloss.backward()
        optimizer.step()

        train_loss += batchloss.item() * values.size(0)
        absolute_errors = torch.abs(prediction - lables)
        total_mae += torch.sum(absolute_errors).item()
        total_rmse += torch.sum((prediction - lables) ** 2).item()

        ss_residual = torch.sum((lables - prediction) ** 2)
        ss_total = torch.sum((lables - torch.mean(lables)) ** 2)
        test_r2 += (1 - (ss_residual / ss_total)).item()


    with torch.no_grad():
        for data in valdataloader:
            values,lables=data
            prediction=model(values).squeeze(1)
            batchloss=critetion(prediction,lables)

            # total_loss_val+=batchloss.item()*values.size(0)
            # total_acc_val+=(prediction-lables).sum().item()
            # total_samples_val += lables.size(0)
            
           
    avg_train_loss = train_loss / len(traindataloader.dataset)
    avg_mae = total_mae / len(traindataloader.dataset)
    avg_rmse = (total_rmse / len(traindataloader.dataset)) ** 0.5        
    avg_val_r2 = test_r2 / len(testdataloader)


    print(f"""Epoch:{i+1}
          
          Train Loss (MSE): {avg_train_loss:.4f} | MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f}| R^2{avg_val_r2:.4f}""")  #val_loss: {total_loss_val:.4f} ,val_accuracy: {total_acc_val:.4f}""")
    
    print("="*20)




Epoch:1
          
          Train Loss (MSE): 355914688102.4000 | MAE: 583663.3143 | RMSE: 596585.8600| R^2-118.5789
Epoch:2
          
          Train Loss (MSE): 355914688570.5143 | MAE: 583663.3071 | RMSE: 596585.8613| R^2-153.8653
Epoch:3
          
          Train Loss (MSE): 355914688570.5143 | MAE: 583663.3143 | RMSE: 596585.8613| R^2-133.4086
Epoch:4
          
          Train Loss (MSE): 355914687166.1714 | MAE: 583663.3171 | RMSE: 596585.8581| R^2-134.3386
Epoch:5
          
          Train Loss (MSE): 355914687166.1714 | MAE: 583663.3214 | RMSE: 596585.8588| R^2-125.4537
Epoch:6
          
          Train Loss (MSE): 355914689506.7429 | MAE: 583663.2943 | RMSE: 596585.8607| R^2-137.5904
Epoch:7
          
          Train Loss (MSE): 355914691379.2000 | MAE: 583663.3100 | RMSE: 596585.8600| R^2-136.4499
Epoch:8
          
          Train Loss (MSE): 355914684825.6000 | MAE: 583663.3043 | RMSE: 596585.8581| R^2-125.9049
Epoch:9
          
          Train Loss (MSE): 355914691